In [1]:
import os
import sys
import pandas as pd
import numpy as np
import glob

In [2]:
base_dir ="/home/yatin/hpchome/phd/comboptnet/CombOptNetJAX/"
os.chdir(base_dir)

In [3]:
def extract_short_key_sudoku(row):
    big_key = row[gkey]
    return ('.'.join(big_key.split('/')[-2:]))

def extract_short_key_rc(row):
    big_key = row[gkey]
    return ('.'.join(big_key.split('/')[1:]))

def add_grpby_columns_rc(tab):
    new_cols = tab[gkey].str.split('/', expand=True)
    new_cols = new_cols.iloc[:,1:]
    new_colnames = ['data','num_cons','seed']
    new_cols.columns = new_colnames
    tab = pd.concat([tab,new_cols], axis=1)
    tab[skey] = tab.apply(extract_short_key_rc, axis=1)
    return tab

grpby_keys_rc = ['data','num_cons']
grp_over_rc = ['seed']
exp_dir_list_rc = ['results/']
es_col_rc = 'eval_acc'

expected_count_rc = 10
json_filename = 'metrics.json'


In [4]:
#version_list =  ['version_0']
dataset= 'rc'


In [5]:
gkey = 'logdir'
total_train_time = 'total_train_time'
total_val_time = 'total_val_time'
epoch_test_time  = 'test_time'
net_train_time = 'total_train_time'
val_acc = 'eval_perfect_accuracy'
test_acc = 'test_perfect_accuracy'
skey = 'skey'
best_epoch = 'best_epoch'


stat_cols = [val_acc,test_acc,best_epoch]
time_cols = [net_train_time,total_val_time,epoch_test_time]

In [6]:
if dataset== 'rc':
    add_grpby_columns = add_grpby_columns_rc
    grpby_keys = grpby_keys_rc
    exp_dir_list = exp_dir_list_rc
    expected_count = expected_count_rc
    grp_over = grp_over_rc
    es_col = es_col_rc
else:
    raise

In [7]:
all_tables = []
for exp_dir in exp_dir_list:
    for fpath in glob.glob(os.path.join(exp_dir,"**"), recursive=True):
        #print(fpath)
        log_file = os.path.join(fpath,json_filename)
        if os.path.exists(log_file):
            this_table = pd.read_json(log_file, orient = 'records',lines = True)
            this_table['logdir'] = fpath
            all_tables.append(this_table)
        #else:
            #print('missing: ', fpath)
    

tab = pd.concat(all_tables)

In [8]:
tab.reset_index(inplace=True,drop=True)

In [9]:
tab = add_grpby_columns(tab)

In [10]:
#extract short key

tab[time_cols] = tab[time_cols]/60.0

In [43]:
test_tab = tab[tab['epoch']  == -1]
last_test_tab = tab[tab['epoch']  == 99]

In [12]:
count_bf = test_tab.groupby(grpby_keys)[grp_over[0]].count().to_frame()

In [13]:
count_bf

seed
data                num_cons      
binary_random       1x16        10
                    2x16        10
                    4x16        10
                    8x16        10
binary_random_50pct 1x16        10
                    2x16        10
                    4x16        10
                    8x16        10
dense_random        1x16        10
dense_random_50pct  1x16        10
                    2x16        10
                    4x16        10
                    8x16        10

In [14]:
if ((count_bf != expected_count).any().any()):
    print('Some exp. ran more than once or not even once. Please check. Picking the last one ...')

In [15]:
    

test_tab = test_tab.groupby(grpby_keys+grp_over).tail(1)

count = test_tab.groupby(grpby_keys)[grp_over[0]].count().to_frame()

assert (count == expected_count).all().all()

all_stats = test_tab.groupby(grpby_keys)[stat_cols+time_cols].agg(['mean','std'])

stats = all_stats.transpose().round(4)

In [20]:
stats

data                       binary_random                             \
num_cons                            1x16     2x16     4x16     8x16   
eval_perfect_accuracy mean        0.9990   0.9840   0.9040   0.6910   
                      std         0.0032   0.0126   0.0762   0.0963   
test_perfect_accuracy mean        0.9793   0.9608   0.8926   0.6697   
                      std         0.0071   0.0132   0.0750   0.0851   
best_epoch            mean       35.0000  83.0000  79.0000  79.0000   
                      std        16.6333  23.5466  14.9071  21.9848   
total_train_time      mean       19.9055  21.8821  26.3395  34.1393   
                      std         0.1577   0.1768   1.5119   2.5666   
total_val_time        mean        0.0782   0.0932   0.1351   0.2311   
                      std         0.0015   0.0035   0.0217   0.0368   
test_time             mean        0.0394   0.0459   0.0668   0.1120   
                      std         0.0005   0.0009   0.0098   0.0176   

data                       binary_random_50pct                             \
num_cons                                  1x16     2x16     4x16     8x16   
eval_perfect_accuracy mean              0.9970   0.9410   0.7540   0.5820   
                      std               0.0067   0.0428   0.1143   0.1070   
test_perfect_accuracy mean              0.9783   0.9285   0.7398   0.5589   
                      std               0.0100   0.0345   0.1245   0.0815   
best_epoch            mean             46.5000  83.5000  94.5000  89.0000   
                      std              17.0375  15.7145   5.5025  10.5409   
total_train_time      mean             10.0260  11.0004  13.1534  17.6738   
                      std               0.0984   0.0661   0.7323   1.3668   
total_val_time        mean              0.0777   0.0924   0.1341   0.2438   
                      std               0.0020   0.0023   0.0177   0.0469   
test_time             mean              0.0379   0.0451   0.0667   0.1165   
                      std               0.0054   0.0011   0.0091   0.0197   

data                       dense_random dense_random_50pct                    \
num_cons                           1x16               1x16     2x16     4x16   
eval_perfect_accuracy mean       0.9250             0.8810   0.6580   0.1590   
                      std        0.0242             0.0360   0.1429   0.1240   
test_perfect_accuracy mean       0.8915             0.8739   0.6448   0.1404   
                      std        0.0358             0.0354   0.1488   0.0854   
best_epoch            mean      76.5000            82.5000  90.5000  95.0000   
                      std       19.3290            16.8408  13.5503   6.1464   
total_train_time      mean      20.1294            10.1298  11.4271  13.5875   
                      std        0.1554             0.0676   0.2086   0.2611   
total_val_time        mean       0.0798             0.0806   0.1042   0.1490   
                      std        0.0022             0.0023   0.0075   0.0100   
test_time             mean       0.0403             0.0405   0.0505   0.0736   
                      std        0.0005             0.0007   0.0026   0.0048   

data                                 
num_cons                       8x16  
eval_perfect_accuracy mean   0.0130  
                      std    0.0157  
test_perfect_accuracy mean   0.0137  
                      std    0.0134  
best_epoch            mean  94.5000  
                      std    6.4334  
total_train_time      mean  16.8576  
                      std    0.7557  
total_val_time        mean   0.2168  
                      std    0.0214  
test_time             mean   0.1130  
                      std    0.0114

In [38]:
test_stats = all_stats[[test_acc,net_train_time, best_epoch]].transpose().round(4)

In [39]:
test_stats

data                       binary_random                             \
num_cons                            1x16     2x16     4x16     8x16   
test_perfect_accuracy mean        0.9793   0.9608   0.8926   0.6697   
                      std         0.0071   0.0132   0.0750   0.0851   
total_train_time      mean       19.9055  21.8821  26.3395  34.1393   
                      std         0.1577   0.1768   1.5119   2.5666   
best_epoch            mean       35.0000  83.0000  79.0000  79.0000   
                      std        16.6333  23.5466  14.9071  21.9848   

data                       binary_random_50pct                             \
num_cons                                  1x16     2x16     4x16     8x16   
test_perfect_accuracy mean              0.9783   0.9285   0.7398   0.5589   
                      std               0.0100   0.0345   0.1245   0.0815   
total_train_time      mean             10.0260  11.0004  13.1534  17.6738   
                      std               0.0984   0.0661   0.7323   1.3668   
best_epoch            mean             46.5000  83.5000  94.5000  89.0000   
                      std              17.0375  15.7145   5.5025  10.5409   

data                       dense_random dense_random_50pct                    \
num_cons                           1x16               1x16     2x16     4x16   
test_perfect_accuracy mean       0.8915             0.8739   0.6448   0.1404   
                      std        0.0358             0.0354   0.1488   0.0854   
total_train_time      mean      20.1294            10.1298  11.4271  13.5875   
                      std        0.1554             0.0676   0.2086   0.2611   
best_epoch            mean      76.5000            82.5000  90.5000  95.0000   
                      std       19.3290            16.8408  13.5503   6.1464   

data                                 
num_cons                       8x16  
test_perfect_accuracy mean   0.0137  
                      std    0.0134  
total_train_time      mean  16.8576  
                      std    0.7557  
best_epoch            mean  94.5000  
                      std    6.4334

In [45]:
    

last_test_tab = last_test_tab.groupby(grpby_keys+grp_over).tail(1)

count = last_test_tab.groupby(grpby_keys)[grp_over[0]].count().to_frame()

assert (count == expected_count).all().all()

last_all_stats = last_test_tab.groupby(grpby_keys)[stat_cols+time_cols].agg(['mean','std'])

last_stats = last_all_stats.transpose().round(4)

In [47]:
last_test_stats = last_all_stats[[test_acc,net_train_time, best_epoch]].transpose().round(4)

In [49]:
last_all_stats

eval_perfect_accuracy            \
                                              mean       std   
data                num_cons                                   
binary_random       1x16                     0.991  0.011972   
                    2x16                     0.969  0.017920   
                    4x16                     0.873  0.074989   
                    8x16                     0.611  0.156166   
binary_random_50pct 1x16                     0.972  0.029740   
                    2x16                     0.927  0.042701   
                    4x16                     0.749  0.121147   
                    8x16                     0.544  0.086178   
dense_random        1x16                     0.886  0.071523   
dense_random_50pct  1x16                     0.841  0.054863   
                    2x16                     0.621  0.129910   
                    4x16                     0.164  0.121491   
                    8x16                     0.013  0.013375   

                             test_perfect_accuracy     best_epoch             \
                                              mean std       mean        std   
data                num_cons                                                   
binary_random       1x16                       NaN NaN       35.0  16.633300   
                    2x16                       NaN NaN       83.0  23.546644   
                    4x16                       NaN NaN       79.0  14.907120   
                    8x16                       NaN NaN       79.0  21.984843   
binary_random_50pct 1x16                       NaN NaN       46.5  17.037540   
                    2x16                       NaN NaN       83.5  15.714466   
                    4x16                       NaN NaN       94.5   5.502525   
                    8x16                       NaN NaN       89.0  10.540926   
dense_random        1x16                       NaN NaN       76.5  19.329023   
dense_random_50pct  1x16                       NaN NaN       82.5  16.840757   
                    2x16                       NaN NaN       90.5  13.550318   
                    4x16                       NaN NaN       95.0   6.146363   
                    8x16                       NaN NaN       94.5   6.433420   

                             total_train_time           total_val_time  \
                                         mean       std           mean   
data                num_cons                                             
binary_random       1x16            19.905500  0.157729       0.078205   
                    2x16            21.882131  0.176837       0.093169   
                    4x16            26.339544  1.511943       0.135145   
                    8x16            34.139310  2.566644       0.231146   
binary_random_50pct 1x16            10.025979  0.098426       0.077691   
                    2x16            11.000351  0.066088       0.092359   
                    4x16            13.153355  0.732275       0.134149   
                    8x16            17.673807  1.366825       0.243799   
dense_random        1x16            20.129375  0.155390       0.079789   
dense_random_50pct  1x16            10.129755  0.067575       0.080648   
                    2x16            11.427135  0.208605       0.104158   
                    4x16            13.587537  0.261114       0.149010   
                    8x16            16.857556  0.755691       0.216764   

                                       test_time      
                                   std      mean std  
data                num_cons                          
binary_random       1x16      0.001493       NaN NaN  
                    2x16      0.003505       NaN NaN  
                    4x16      0.021651       NaN NaN  
                    8x16      0.036822       NaN NaN  
binary_random_50pct 1x16      0.001988       NaN NaN  
                    2x16      0.002303       NaN NaN  
                    4x16      0.017671     

In [25]:
time_tab = tab.iloc[tab.groupby(skey)[es_col].idxmax().dropna()]

(130, 29)

In [26]:
time_tab[time_tab['epoch'] == -1]

,train_loss,train_acc,train_perfect_accuracy,train_match_uncon_accuracy,train_match_uncon_perfect_accuracy,train_time,epoch,total_train_time,total_val_time,is_best,...,test_acc,test_perfect_accuracy,test_match_uncon_accuracy,test_match_uncon_perfect_accuracy,test_time,logdir,data,num_cons,seed,skey


In [27]:
count_time = time_tab.groupby(grpby_keys)[grp_over[0]].count().to_frame()

In [28]:
count_time

seed
data                num_cons      
binary_random       1x16        10
                    2x16        10
                    4x16        10
                    8x16        10
binary_random_50pct 1x16        10
                    2x16        10
                    4x16        10
                    8x16        10
dense_random        1x16        10
dense_random_50pct  1x16        10
                    2x16        10
                    4x16        10
                    8x16        10

In [34]:
time_stats = time_tab.groupby(grpby_keys)[time_cols].agg(['mean','std']).round(4)

In [35]:
time_stats = time_stats[[net_train_time]].transpose()

In [42]:
time_stats

data                  binary_random                             \
num_cons                       1x16     2x16     4x16     8x16   
total_train_time mean        7.2143  18.3871  21.1118  27.1549   
                 std         3.2833   5.0971   4.2828   6.5685   

data                  binary_random_50pct                            \
num_cons                             1x16    2x16     4x16     8x16   
total_train_time mean              4.8027  9.3080  12.5446  15.8540   
                 std               1.6839  1.6986   0.7191   1.5261   

data                  dense_random dense_random_50pct                    \
num_cons                      1x16               1x16     2x16     4x16   
total_train_time mean      15.6314             8.4777  10.4784  13.0439   
                 std        3.8572             1.6680   1.5389   0.8267   

data                            
num_cons                  8x16  
total_train_time mean  16.0980  
                 std    1.3764

In [37]:
#extract the relevant rows

In [ ]:
result = pd.concat([count.transpose(),stats,time_stats])

In [ ]:
test_result = pd.concat([count.transpose().droplevel('ver',axis=1),
                         test_stats.droplevel('ver',axis=1),
                         time_stats.droplevel('ver',axis=1)],axis=0)

In [ ]:
"""
test_result = pd.concat([count.transpose(),
                         test_stats,
                         time_stats],axis=0)
                         
"""


In [ ]:
test_result

In [ ]:
time_stats.droplevel('ver',axis=1)

In [ ]:
test_result